<a href="https://colab.research.google.com/github/uervitonsantos/Otimizacao-Politicaca-Manutencao-Preventiva-Imperfeita/blob/main/Algoritmo_PSO_para_otimiza%C3%A7%C3%A3o_de_custo_de_manuten%C3%A7%C3%A3o_preventiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
%%capture
pip install pyswarms

In [60]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time
from pyswarms.single.global_best import GlobalBestPSO
import pyswarms as ps
from pyswarms.utils.search import RandomSearch
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)
import plotly.graph_objects as go
from pyswarms.utils.plotters.formatters import Designer
from pyswarms.utils.plotters.formatters import Mesher
from mpl_toolkits.mplot3d import Axes3D

In [28]:
MAX_VALUE_CUSTO = 10000
fator_melhoria = 'expo'

In [29]:
# Custos MC e MP
CMC = 4000
CMP = 2000

In [30]:
class IntensidadeFalha:
    # Este método vai inicializar cada objeto criado a partir desta classe
    # O nome deste método é "inicio"
    # (self) é uma referência a cada atributo de um objeto criado a partir desta classe
    # Os atributos de cada objeto criados a partir desta classe.
    # O self indica que estes são atributos dos objetos
    def __init__(self):
        self.fator = None
        self.lamb = None
        self.beta = None
        self.theta = None
        self.sz = None
        self.T = None
        self.u = None

    def inicio(self, T, fator, solucao):
        self.fator = fator
        self.lamb = solucao[0]
        self.beta = solucao[1]
        self.theta = solucao[2]
        self.sz = int(T[-1])
        self.T = T

    def create(self):
        self.u = []
        # Reducao de Idade com fator de melhoria linear
        for i, tc in enumerate(self.T[1:], start=1):
            for tt in range(int(self.T[i-1]), int(tc)+1):
                b = tt - self.fator[0][i-1] * self.T[i-1]
                self.u.append((1 / (self.lamb ** self.beta)) * self.beta * b ** (self.beta - 1))

                if tt == tc:
                    self.u.append(float('nan'))
            break
    # plota e salva a figura da função de intensidade de falha
    def plotar(self):
        fig = plt.figure(figsize=(20, 5))
        plt.plot(self.u)

        for xc in self.T:
            plt.axvline(x=xc, color='r', linestyle='--')

        plt.xlabel("Tempo (dias)")
        plt.ylabel("Função intensidade de falha")
        plt.xlim(0)
        plt.grid(True)
        plt.show()

In [31]:
# Define a função de melhoria constante
def const(ck, m, s, theta):
    fator = [[theta] * ck] * m
    return fator

In [32]:
# Define a função de melhoria linear
def linear(ck, m, s, theta):
    fator = []
    for k in range(0, m, 1):
        linha = [0]
        for j in range(0, ck, 1):
            linha.append(s[k][j] * theta)
        fator.append(linha)
    return fator

In [33]:
 # Define a função de melhoria exponencial
def expo(ck, m, s, theta):
    fator = []
    for k in range(0, m, 1):
        linha = []
        for j in range(0, ck, 1):
            linha.append(1.0 - np.exp(-s[k][j] * theta))
        fator.append(linha)
    return fator

In [34]:
# Define a função de melhoria potência
def pot(ck, m, s, theta):
    fator = []
    for k in range(0, m, 1):
        linha = []
        for j in range(0, ck, 1):
            linha.append(s[k][j] ** theta)
        fator.append(linha)
    return fator

In [35]:
# Define a função de melhoria para manutenção perfeita
def mpp(ck, m, s, theta):
    fator = []
    for k in range(0, m, 1):
        linha = []
        for j in range(0, ck, 1):
            linha.append(1)
        fator.append(linha)
    return fator

In [36]:
# Executa a função de melhoria com base no parâmetro fornecido
def execute_function(fator_melhoria, ck, m, s, theta):
    return {
        'const': lambda: const(ck, m, s, theta),
        'linear': lambda: linear(ck, m, s, theta),
        'expo': lambda: expo(ck, m, s, theta),
        'pot': lambda: pot(ck, m, s, theta),
        'mpp': lambda: mpp(ck, m, s, theta),
    }[fator_melhoria]()

In [37]:
# Função custo de manutenção baseada em severidade
def custo_mp(s, CMP):
    dist = {0.25: 2000, 0.5: 4000, 0.75: 12000, 1.0: 20000}
    cmp_f = dist[s]
    return cmp_f

In [38]:
# Leitura dos dados de tempo: exemplo tese Marcos Coque Jr.
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DateSet/data.csv', sep=";")
df.head()

,t,k,c,n,s,T
0,90,1,1,4.0,0.4,0.0
1,110,1,1,0.0,0.5,150.0
2,125,1,1,9.0,0.9,250.0
3,135,1,1,2.0,0.2,500.0
4,295,1,3,2.0,0.4,600.0


In [39]:
def objective_st(x):

    # Criação de variáveis para os parâmetros
    lamb = x[0]
    beta = x[1]
    theta = x[2]

    # Cálculo da intensidade de falha
    # Cria as variaveis
    # tempos de falhas
    t = df['t']
    # indice do sistema
    k = df['k']
    m = k.drop_duplicates().shape[0]
    # Numero de ciclos de manutenção
    ck = df['c'].max()
    # Numero de falhas e severidade das manutenções preventivas em cada sistema
    n = []
    s = []
    for k in range(m):
        n.append(list(df['n'][k * ck:(k + 1) * ck]))
        s.append(list(df['s'][k * ck:(k + 1) * ck]))

    # Tempos das MP
    T = list(df['T'][:m])

    # Cria alguns cenários de teste
    beta = 5

    solucao = [lamb, beta, theta]

    # Cria a instância da classe IntensidadeFalha
    falha = IntensidadeFalha()

    # Inicializa a instância com os valores necessários
    falha.inicio(T, execute_function(fator_melhoria, ck, m, s, theta), solucao)

    # Cria os dados da intensidade de falha
    falha.create()

    # Cálculo do custo total
    custo_total = 0
    for ui in falha.u:
        if not np.isnan(ui).any(): #  Para verificar se há algum valor NaN no array
            custo_total += CMP + (CMC * (1 - np.exp(-lamb * ui)))

    custo_total = np.minimum(custo_total, MAX_VALUE_CUSTO)  # Corrigido para limitar o valor ao máximo permitido

    return custo_total

In [80]:
num_particles = 50
max_iterations = 1000

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = GlobalBestPSO(n_particles=num_particles, dimensions=50, options=options)

# Inicializar current_cost e pbest_cost com valores infinitos
optimizer.swarm.current_cost = np.full(optimizer.swarm.position.shape[0], np.inf)
optimizer.swarm.pbest_cost = np.full(optimizer.swarm.position.shape[0], np.inf)
best_cost, best_position = optimizer.optimize(objective_st, iters=max_iterations)
pbest_cost = optimizer.swarm.pbest_cost

print("                              ")
print("                              ")
print("*************************************************************************")
print("Número total de iterações:", max_iterations)
print("Número de Particulas:", num_particles)
print("                              ")
print("Melhor posição:", best_position[0])
print("Melhor custo:", best_cost)
print("Melhor custo pessoal (pbest) de cada partícula:", pbest_cost[0])
print("*************************************************************************")

2023-06-30 01:38:16,250 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=1e+4
2023-06-30 01:38:29,777 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 10000.0, best pos: [0.65463739 0.37801986 0.25634358 0.21058133 0.33974695 0.03809459
 0.82355318 0.68552883 0.93341363 0.5194402  0.61529144 0.25027878
 0.77691369 0.10157146 0.91825866 0.83307425 0.47223637 0.84141459
 0.23486942 0.06190796 0.51705103 0.4828888  0.61483382 0.72117426
 0.92632167 0.96190478 0.92748007 0.67806191 0.50586721 0.50398034
 0.42854766 0.46828937 0.52572098 0.60221765 0.13306792 0.38089186
 0.22290788 0.73582523 0.4753056  0.26843162 0.24804135 0.66645707
 0.25685788 0.93247978 0.09032322 0.62179613 0.28517868 0.15772992
 0.63577996 0.46212874]


                              
                              
*************************************************************************
Número total de iterações: 1000
Número de Particulas: 50
                              
Melhor posição: 0.654637388975709
Melhor custo: 10000.0
Melhor custo pessoal (pbest) de cada partícula: 10000.0
*************************************************************************


In [ ]:
# Preprocessamento dos dados
m = Mesher(func=objective_st)
pos_history_3d = m.compute_history_3d(optimizer.pos_history)

# Configuracoes de exibicao do grafico
limits = [(-1, 1)] * pos_history_3d[0].shape[1]
labels = ['x-axis', 'y-axis', 'z-axis']

# Criar a figura e adicionar a superficie
fig = go.Figure()
fig.add_surface(x=pos_history_3d[:, :, 0], y=pos_history_3d[:, :, 1], z=pos_history_3d[:, :, 2])

# Ajustar as configuracoes do layout
fig.update_layout(scene=dict(xaxis_title=labels[0], yaxis_title=labels[1], zaxis_title=labels[2]))
fig.update_layout(scene=dict(xaxis=dict(range=limits[0]), yaxis=dict(range=limits[1]), zaxis=dict(range=limits[2])))

# Adicionar a marca (minimo)
fig.add_trace(go.Scatter3d(x=[best_position[0]], y=[best_position[1]], z=[best_position[2]],
                           mode='markers', marker=dict(color='red', size=5)))
# Exibir o grafico
fig.show()